# 05 Functions

**PyJudo** supports injecting dependencies into functions. This is achieved with the `@container.inject` decorator.  

In some circumstances, it may be useful to add dependencies to a function or method, rather than in a class constructor.

**PyJudo** supports injecting dependencies into vanilla functions, bound methods, class methods and static methods.

In [1]:
from abc import ABC, abstractmethod

from pyjudo import ServiceContainer

Lets take a look at some examples:

In [2]:
# Interfaces
class IFoo(ABC):
    @abstractmethod
    def print_text(self): ...

class IBar(ABC):
    @abstractmethod
    def print_text(self): ...

class IBaz(ABC):
    @abstractmethod
    def print_text(self): ...

# Implementations
class Foo(IFoo):
    def __init__(self, text:str = "text"):
        self.text = text

    def print_text(self):
        print(self.text)

class Bar(IBar):
    def __init__(self, text:str = "text"):
        self.text = text

    def print_text(self):
        print(self.text)

class Baz(IBaz):
    def __init__(self, foo:IFoo, bar:IBar):
        self.foo = foo
        self.bar = bar

    def print_text(self):
        self.foo.print_text()
        self.bar.print_text()
    
    def dispose(self):
        print("Disposing Baz")

In [3]:
# Serup a container
container = ServiceContainer()
container.add_singleton(IFoo, Foo)
container.add_transient(IBar, Bar)
container.add_scoped(IBaz, Baz)

In [4]:
# Lets get some services, and notably create the singleton IFoo with overwritten text
foo = container.get(IFoo, text="override")
bar = container.get(IBar, text="override")

# Define a function that takes some arguments, and injects the IBaz service
@container.inject
def something(x: int, baz: IBaz, y: int):
    print(f"{x=}, {y=}")
    baz.print_text()

# IBaz is scoped, so we need to create a scope to get a new instance
with container.create_scope():
    something(x=2, y=1)

x=2, y=1
override
text
Disposing Baz


We can see in this example that we didn't explicitly pass an argument for `baz` in `something(...)`; the `ServiceContainer` resolved it for us, and also took care of its disposal.

Whilst a little out of the norm; we may also want to inject services into methods. We can inject services into instance (bound), class and static methods using the same `@container.inject` syntax:

In [5]:
class Qux:
    @container.inject
    def do_a_foo(self, foo: IFoo):
        foo.print_text()

    @container.inject
    @classmethod
    def do_a_bar(cls, bar: IBar):
        bar.print_text()

    @container.inject
    @staticmethod
    def do_a_static_bar(baz: IBaz):
        baz.print_text()

In [6]:
# Bound method
print("Bound method injection")
q = Qux()
q.do_a_foo()

# Class method
print("\nClassmethod injection")
Qux.do_a_bar()

# Static method
print("\nStaticmethod injection")
with container.create_scope():
    Qux.do_a_static_bar()

Bound method injection
override

Classmethod injection
text

Staticmethod injection
override
text
Disposing Baz


Like the previous example, we can see that the `ServiceContainer` resolves the dependencies for the methods, adhering to the lifetimes we specified for the services during registration.

Overall, function injection gives us more flexibility for injecting dependencies but keeps the interface simple 😎